# Mount Drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install Libraries and Import

In [4]:
%%capture
!pip install clodsa
!pip install -q -U albumentations

In [5]:
!pip uninstall imgaug

Uninstalling imgaug-0.2.9:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/imgaug-0.2.9.dist-info/*
    /usr/local/lib/python3.7/dist-packages/imgaug/*
Proceed (y/n)? y
  Successfully uninstalled imgaug-0.2.9


In [6]:
%%capture
!pip install git+https://github.com/aleju/imgaug.git

In [7]:
from matplotlib import pyplot as plt
from clodsa.augmentors.augmentorFactory import createAugmentor
from clodsa.transformers.transformerFactory import transformerGenerator
from clodsa.techniques.techniqueFactory import createTechnique
import random
import cv2
import albumentations as A
import imageio
import imgaug as ia
import imgaug.augmenters as iaa
import os 

# CLoDSA

#### This notebook applies different augmentation techniques to the original 20 sample images and saves them in a directory. 

## Dropout

In [8]:
def apply_dropout(input_path, output_path, percentages):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  for percentage in percentages:
    dropout = createTechnique("dropout", {"percentage" : percentage})
    augmentor.addTransformer(transformer(dropout))
  
  augmentor.applyAugmentation()
  print("Dropout results were saved given directory.")

In [10]:
# 0.05, 0,1 should be in seperate folders
apply_dropout("/content/drive/MyDrive/train",
               "/content/drive/MyDrive/dropout",
               [0.1])

Dropout results were saved given directory.


## Gamma Correction

In [12]:
def apply_gamma_correction(input_path, output_path, gammas):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  for gamma in gammas:
    gamma_t = createTechnique("gamma", {"gamma" : gamma})
    augmentor.addTransformer(transformer(gamma_t))
  
  augmentor.applyAugmentation()
  print("Gamma correction results were saved given directory.")

In [13]:
#1.5, 2 should be in seperate folders
apply_gamma_correction("/content/drive/MyDrive/train",
               "/content/drive/MyDrive/gamma_correction",
               [0.5])

Gamma correction results were saved given directory.


## White Noise

In [14]:
def apply_white_noise(input_path, output_path, sd):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  for sigma in sd:
    white_noise = createTechnique("gaussian_noise", {"mean" : 0,"sigma":sigma})
    augmentor.addTransformer(transformer(white_noise))
  
  augmentor.applyAugmentation()
  print("White noise results were saved given directory.")

In [16]:
apply_white_noise("/content/drive/MyDrive/train",
               "/content/drive/MyDrive/white_noise",
               [10])

White noise results were saved given directory.


## Equalize Histogram

In [17]:
 def apply_eqhisto(input_path, output_path):
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)
  equalize = createTechnique("equalize_histogram",{})
  augmentor.addTransformer(transformer(equalize)) 
  augmentor.applyAugmentation()

  print("Equalize histogram results were saved given directory.")

In [18]:
apply_eqhisto("/content/drive/MyDrive/train", 
              "/content/drive/MyDrive/eq_hist")

Equalize histogram results were saved given directory.


## Blurring

In [19]:
def aug_blurring(input_path, output_path, blurr:list):
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})
  transformer = transformerGenerator(PROBLEM)

  for ker in blurr:
    blur = createTechnique("blurring", {"kernel" : ker})
    augmentor.addTransformer(transformer(blur))
    print("Blurring for kernel = {} is done".format(ker))
  
  augmentor.applyAugmentation()
  print("Augmentation results were saved given directory.")

In [20]:
#3, 5 should be in seperate folders
aug_blurring("/content/drive/MyDrive/train",
             "/content/drive/MyDrive/blurring", [5])

Blurring for kernel = 5 is done
Augmentation results were saved given directory.


## Elastic Deformation

In [21]:
def apply_elastic_deformation(input_path, output_path, alpha = 5, sigma = 0.05):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("elastic", {"alpha" : alpha, "sigma" : sigma})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Elastic deformation results were saved given directory.")

In [22]:
apply_elastic_deformation('/content/drive/MyDrive/train',
                          '/content/drive/MyDrive/elastic_transform')

Elastic deformation results were saved given directory.


## Flipping

In [23]:
def apply_flipping(input_path, output_path, flip):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("flip", {"flip" : flip})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Flipping results were saved given directory.")

In [24]:
apply_flipping('/content/drive/MyDrive/train',
                '/content/drive/MyDrive/train/flipping1', 1)

Flipping results were saved given directory.


## Shearing

In [25]:
def apply_shearing(input_path, output_path, a = 0.5):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("shearing", {"a" : a})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Shearing results were saved given directory.")

In [26]:
apply_shearing('/content/drive/MyDrive/train',
                '/content/drive/MyDrive/shearing',a = 0.3)

Shearing results were saved given directory.


## Sharpen

In [27]:
def apply_sharpen(input_path, output_path):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("sharpen", {})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Sharping results were saved given directory.")

In [28]:
apply_sharpen('/content/drive/MyDrive/train',
                '/content/drive/MyDrive/sharpen')

Sharping results were saved given directory.


## Raise Saturation

In [29]:
def apply_raise_satur(input_path, output_path, power):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - angels -> this should be a list that includes the angels that you want to rotate images and labels.
  '''
  PROBLEM = "semantic_segmentation"
  ANNOTATION_MODE = "folders"
  INPUT_PATH = input_path + "/"
  GENERATION_MODE = "linear"
  OUTPUT_MODE = "folders"
  OUTPUT_PATH= output_path + "/"
  LABELS_EXTENSION = ".png"

  augmentor = createAugmentor(PROBLEM,ANNOTATION_MODE,OUTPUT_MODE,GENERATION_MODE,INPUT_PATH,{"outputPath":OUTPUT_PATH,"labelsExtension":LABELS_EXTENSION})

  transformer = transformerGenerator(PROBLEM)

  rotate = createTechnique("raise_saturation", {"power" : power})
  augmentor.addTransformer(transformer(rotate))

  augmentor.applyAugmentation()
  print("Raise saturation results were saved given directory.")

In [30]:
apply_raise_satur('/content/drive/MyDrive/train',
                '/content/drive/MyDrive/saturation4', 0.5)

Raise saturation results were saved given directory.


# IMGAUG

## Rotation

In [31]:
input_path = '/content/drive/MyDrive/train'
output_path = '/content/drive/MyDrive/deneme8'

In [34]:
def rotation(degree, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.Rotate((degree)),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'rotat'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'rotat'+ '_' + str(indx) + '.png', i)

  print("Rotation results were saved given directory.")

In [35]:
rotation(30,input_path,output_path,20)

Rotation results were saved given directory.


## JPEG Compression

In [36]:
import imageio
import imgaug as ia
import os 

def apply_jpeg_compression(input_path, output_path, degrees):
  '''
  - input_path -> folder tree should be like that : input_path-->|
                                                                 |-> images
                                                                 |-> labels

  - output_path -> this is the directory that will store the results

  - degrees -> this should be a list that includes the degrees that you want to compress images and labels (must be between 0 and 100).
  '''

  images = []
  labels = []

  for img_path in range(20):
    img = imageio.imread(input_path + 'images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + 'labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path) 

  for degree in degrees:
    aug = ia.augmenters.JpegCompression(compression=degree)
    
    images_aug = aug.augment_images(images=images)
    for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + 'images/' + str(degree) + '_' + str(indx) + '.png', i)

    labels_aug = aug.augment_images(images=labels)
    for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + 'labels/' + str(degree) + '_' + str(indx) + '.png', i)

  print("JPEG Compression results were saved given directory.")

In [39]:
# 25,50,75 should be in seperate folders

apply_jpeg_compression("/content/drive/MyDrive/train/",
                       "/content/drive/MyDrive/jpeg_compression25/",
                        [25])

JPEG Compression results were saved given directory.


## Zoom In / Out

In [45]:
input_path = '/content/drive/MyDrive/train'
output_path = '/content/drive/MyDrive/deneme7'

In [46]:
def zoom(zoom_amount, input_path, output_path, image_count):
  images = []
  labels = []
  ia.seed(1)
  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.Affine(
              scale={"x": (zoom_amount), "y": (zoom_amount)}
          ),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'zoom'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'zoom'+ '_' + str(indx) + '.png', i)

  print("Zoom results were saved given directory.")

In [47]:
zoom(0.8, input_path, output_path, 20)

Zoom results were saved given directory.


## Shear X/Y

In [48]:
input_path = '/content/drive/MyDrive/train'
output_path = '/content/drive/MyDrive/deneme6'

In [49]:
def shearX(shear_amount, input_path, output_path, image_count):
  images = []
  labels = []
  ia.seed(1)
  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.ShearX((shear_amount)),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'shear'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'shear'+ '_' + str(indx) + '.png', i)

  print("Shear results were saved given directory.")

In [50]:
def shearY(shear_amount, input_path, output_path, image_count):
  images = []
  labels = []
  ia.seed(1)
  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
      [

          iaa.ShearY((shear_amount)),

      ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'shear'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + '/labels/'  + 'shear'+ '_' + str(indx) + '.png', i)

  print("Shear results were saved given directory.")

In [51]:
shearX(20, input_path, output_path, 20)

Shear results were saved given directory.


## Contrast

In [52]:
%%capture
!pip install imagecorruptions

In [53]:
input_path = '/content/drive/MyDrive/train'
output_path = '/content/drive/MyDrive/deneme5'

In [54]:
def contrast(severity, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + '/images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + '/labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
    [

        iaa.imgcorruptlike.Contrast(severity=severity),

    ]
  )

  images_aug = seq(images=images)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + '/images/'  + 'contrast'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels):
      imageio.imwrite(output_path + '/labels/'  + 'contrast'+ '_' + str(indx) + '.png', i)

  print("Contrast results were saved given directory.")

In [55]:
contrast(2,input_path, output_path, image_count= 20)

Contrast results were saved given directory.



## Shift X Y

In [56]:
import numpy as np
import imgaug as ia
import imgaug.augmenters as iaa
import imageio
import os

input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train/'
output_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/aug/'

shift_amount = -100 #pixel
image_count  = 20

In [ ]:
def shiftX(shift_amount, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + 'images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + 'labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
    [

        iaa.TranslateX(px=(shift_amount))

    ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + 'images/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + 'labels/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  print("Shift results were saved given directory.")

In [ ]:
def shiftY(shift_amount, input_path, output_path, image_count):
  images = []
  labels = []

  for img_path in range(image_count):
    img = imageio.imread(input_path + 'images/' + str(img_path) + '.png')
    images.append(img) 

    lbl = imageio.imread(input_path + 'labels/' + str(img_path) + '.png')
    labels.append(lbl)
  
  seq = iaa.Sequential(
    [

        iaa.TranslateY(px=(shift_amount))

    ]
  )

  images_aug = seq(images=images)
  labels_aug = seq(images=labels)

  path = os.path.join(output_path, 'images') 
  os.mkdir(path) 

  path = os.path.join(output_path, 'labels') 
  os.mkdir(path)

  for indx, i in enumerate(images_aug):
      imageio.imwrite(output_path + 'images/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  for indx, i in enumerate(labels_aug):
      imageio.imwrite(output_path + 'labels/'  + 'shifted'+ '_' + str(indx) + '.png', i)

  print("Shift results were saved given directory.")

In [ ]:
shiftX(shift_amount, input_path, output_path, image_count)

# Albumentations



1.   Create your main folder in your path
2.   Change the paths inside the code properly
3.   Choose the method from [here](https://albumentations.ai/docs/examples/example_kaggle_salt/)
4.   Change the part between hashtags (####) with this method
5.   Output will be in your main path



In [ ]:
import os
import cv2

output_folder_name = 'deneme'

main_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data'
input_path = '/content/drive/MyDrive/AI_Projects/STARE/png_data/train'

original_height, original_width = 608, 704

In [ ]:
print(original_height)
print(original_width)

In [ ]:
def albumentation(output_folder_name, main_path, original_height, original_width, input_path):

  '''
    - output_folder_name : you should give just the name of the output folder, it will be created by function
    - main_path : the folder that output folder will be created and results will be saved
    - input_path : the folder that includes images and labels in seperate folders
  '''

  os.mkdir(main_path + '/'+ output_folder_name)
  os.mkdir(main_path + '/'+ output_folder_name +'/images')
  os.mkdir(main_path + '/'+ output_folder_name +'/labels')

  for img in sorted(os.listdir(input_path + '/images')):

    image = cv2.imread(input_path +'/images/' + img, 0)
    mask  = cv2.imread(input_path +'/labels/' + img, 0)
    
    ##############################################################
    aug = A.Compose([
      A.OneOf([
          A.RandomSizedCrop(min_max_height=(50, 101), height=original_height, width=original_width, p=0.5),
          A.PadIfNeeded(min_height=original_height, min_width=original_width, p=0.5)
      ], p=1),    
      A.VerticalFlip(p=0.5),              
      A.RandomRotate90(p=0.5),
      A.OneOf([
          A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03, p=0.5),
          A.GridDistortion(p=0.5),
          A.OpticalDistortion(distort_limit=2, shift_limit=0.5, p=1)                  
          ], p=0.8),
      A.CLAHE(p=0.8),
      A.RandomBrightnessContrast(p=0.8),    
      A.RandomGamma(p=0.8)])
    ##############################################################

    augmented = aug(image=image, mask=mask)

    image = augmented['image']
    mask = augmented['mask']

    cv2.imwrite(main_path +'/'+ output_folder_name +'/images/' + img, image)
    cv2.imwrite(main_path +'/' + output_folder_name +'/labels/' + img, mask)

In [ ]:
albumentation(output_folder_name, main_path, original_height, original_width, input_path)

# Append One Directory to Another

In [ ]:
import os, shutil

def merge_augmentations(augment_dir, output_dir, list_of_aug_files):
  '''
    - augment_dir         (string)   : The path which has subfolders that are augmentation folders.
    - output_dir          (string)   : The path you want to put all augmentations.
    - list_of_aug_files   (list)     : List of names which contains augmentations.
  '''
  
  os.mkdir(output_dir + '/images')
  os.mkdir(output_dir + '/labels')
  

  for folder in list_of_aug_files:
    if folder != 'train':
      for file in sorted(os.listdir(augment_dir + '/' + folder + '/images')):
        shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + folder + '_' + file.split("_")[-1].split('.')[0] + '.png')
        shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + folder + '_' + file.split("_")[-1].split('.')[0] + '.png')
        #shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + file)
        #shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + file)
    else:
      for file in sorted(os.listdir(augment_dir + '/' + folder + '/images')):
        shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + file.split("_")[-1].split('.')[0] + '.png')
        shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + file.split("_")[-1].split('.')[0] + '.png')
        #shutil.copy(augment_dir + '/' + folder + '/images/' + file, output_dir + '/images/' + file)
        #shutil.copy(augment_dir + '/' + folder + '/labels/' + file, output_dir + '/labels/' + file)

    print(folder + ' folder has been merged...')
    print('Number of images in output: ' + str(len(os.listdir(output_dir + '/images'))))
  
  print('Merging is done successfully!')

In [ ]:
merge_augmentations('/content/drive/MyDrive/AI_Projects/STARE/png_data',
                    '/content/drive/MyDrive/AI_Projects/STARE/combinations_png/B', 
                    ['train', 
                     'flipping0', 
                     'flipping1']

# Experimental Style Transfer Part

#### This part was not included in the final training and testing tasks

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install -q 'scipy<=1.2.1'  # scipy.misc.imread is deprecated in later versions

In [ ]:
!git clone https://github.com/titu1994/Neural-Style-Transfer.git

Cloning into 'Neural-Style-Transfer'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 1400 (delta 1), reused 3 (delta 0), pack-reused 1393
Receiving objects: 100% (1400/1400), 68.16 MiB | 41.74 MiB/s, done.
Resolving deltas: 100% (820/820), done.


In [ ]:
cd Neural-Style-Transfer/

/content/Neural-Style-Transfer


In [ ]:
LABEL_PATH   = "/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/train/labels"
IMG_PATH     = "/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/train/images"
RESULT_PATH  = "/content/drive/MyDrive/AI_Projects/retina_DRIVE/png_data/style_transfer"

In [ ]:
import numpy as np
import os
from skimage.metrics import structural_similarity
import cv2
import numpy as np

In [ ]:
def find_distances():
  distances = np.zeros((20,20),dtype=float)
  for i in sorted(os.listdir(IMG_PATH)):
    for j in sorted(os.listdir(IMG_PATH)):
      a = cv2.imread(IMG_PATH + '/' + i, 0)
      b = cv2.imread(IMG_PATH + '/' + j, 0)

      score, diff = structural_similarity(a, b, full=True)
      distances[int(i.split('.')[0])][int(j.split('.')[0])] = float(score)

  return distances

In [ ]:
def style_transfer(n_of_iter = 10, content_weight = 0.025, style_weight = 1):
  distances = np.zeros((20,20))
  distances = find_distances()

  result_img = RESULT_PATH + '/images'
  result_label = RESULT_PATH + '/labels'

  os.mkdir(result_img)
  os.mkdir(result_label)

  for i in sorted(os.listdir(IMG_PATH)):
    furthest = distances[int(i.split('.')[0])].argmin()
    furthest_value = distances[int(i.split('.')[0])].min()
    os.system('python INetwork.py "'+ IMG_PATH + '/' + i +'" "'+ IMG_PATH + '/' + str(furthest) + '.png' +'" "'+ result_img + '/'+ i.split('.')[0] + '" --image_size 584 --num_iter '+ str(n_of_iter) +' --pool_type "max" --model "vgg19" --content_weight '+ str(content_weight) +' --style_weight ' + str(style_weight))
    os.rename(result_img + '/' + i.split('.')[0] + '_style.png', result_img + '/style_' + i.split('.')[0] + '.png')
    
    a = cv2.imread(IMG_PATH + '/' + i, 0)
    b = cv2.imread(result_img + '/style_' + i , 0)
    score, diff = structural_similarity(a, b, full=True)
    print("style_" + i + " saved successfully.\nFurthest image from " + i + " is " + str(furthest) + " with similarity: " + str(furthest_value) + "\nSimilarity between " + i + " and styled one is: " + str(score) + "\n---------------------------------------------------------------------") 

  os.system("rsync -a "+ LABEL_PATH + "/ " + result_label)
  for i in sorted(os.listdir(result_label)):
    os.rename(result_label + '/' + i, result_label + '/style_' + i.split('.')[0] + '.png')
  print('Label folder successfully copied from LABEL_PATH.')


In [ ]:
style_transfer(n_of_iter = 10, content_weight = 0.025, style_weight = 1)

style_0.png saved successfully.
Furthest image from 0.png is 13 with similarity: 0.6581827873570326
Similarity between 0.png and styled one is: 0.916427185561389
---------------------------------------------------------------------
style_1.png saved successfully.
Furthest image from 1.png is 13 with similarity: 0.6545721609734262
Similarity between 1.png and styled one is: 0.9423020593559593
---------------------------------------------------------------------
style_10.png saved successfully.
Furthest image from 10.png is 13 with similarity: 0.6619912221219528
Similarity between 10.png and styled one is: 0.9374854737404626
---------------------------------------------------------------------
style_11.png saved successfully.
Furthest image from 11.png is 13 with similarity: 0.6916511963871375
Similarity between 11.png and styled one is: 0.9240754506464935
---------------------------------------------------------------------
style_12.png saved successfully.
Furthest image from 12.png is 